In [2]:
from spacepy import pycdf
import pandas as pd
import os
from datetime import datetime
import glob
from pathlib import Path
from cdflib import CDF, cdfepoch
import pandas as pd
import glob

In [ ]:
# Renamed the files to be named from start date-time to end date-time
for file in cdf_files:
    try:
        cdf = cdflib.CDF(file)

        global_atts = cdf.globalattsget()

        start_cdf = global_atts['Observation_start_time'][0]
        end_cdf = global_atts['Observation_end_time'][0]

        start_time = cdfepoch.to_datetime(start_cdf)
        end_time = cdfepoch.to_datetime(end_cdf)

        start_str = str(start_time[0])
        end_str = str(end_time[0])

        new_name = f"{start_str}_{end_str}.cdf"
        
        if os.path.exists(new_name):
            print(f"Warning: {new_name} already exists. Skipping {file}")
            continue
            
        os.rename(file, new_name)
        print(f"Renamed: {file} -> {new_name}")
        
        # Close the CDF file
        cdf.close()
        
    except Exception as e:
        print(f"Error processing {file}: {e}")
        continue

print("Renaming complete!")

In [ ]:
# Converted downloaded .cdf files to .csv format

directory = "Directory/containing/downloaded/bulk/cdf/files/from/SWIS/website"

for filename in os.listdir(directory):
    cdf_file = os.path.join(directory, filename) 
    cdf_file = pycdf.CDF(cdf_file)

    data_dict = {}
    for var in cdf_file:
        try:
            # Flatten the variable if it's multidimensional
            data = cdf_file[var][:]
            if hasattr(data[0], '__len__'):
                print(f"Skipping multidimensional variable: {var}")
                continue
            data_dict[var] = data
        except Exception as e:
            print(f"Skipping {var}: {e}")

    # Convert to pandas DataFrame
    df = pd.DataFrame(data_dict)
    # Save as CSV
    df.to_csv(f'/Users/vishrutgupta/Desktop/ISRO_bah_2025/SWIS_renamed/csv_full/{filename}.csv', index=False)

print("Conversion complete")


In [ ]:
# Merged CACTus data with SWIS files

def check_for_file(date):
    directory = "/Users/vishrutgupta/Desktop/ISRO_bah_2025/merged_data"
    # os.rmdir(directory)
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if (date[0] == filename[:4]) and (date[1] == filename[5:7]) and (date[2] == filename[8:10]):
            df = pd.read_csv(filepath)
            os.remove(filepath)
            return filename, df

    directory = "/Users/vishrutgupta/Desktop/ISRO_bah_2025/SWIS_renamed/csv_full"
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if (date[0] == filename[:4]) and (date[1] == filename[5:7]) and (date[2] == filename[8:10]):
            df = pd.read_csv(filepath)
            return filename, df        
    return None, None


# Extract CACTus data from the net from May 2024 till July 2025
cact_data = pd.read_csv('CACTus_data/all_cme_data.csv')

# Loop through row-wise data from cactus file and merge with SWIS data
for index, row in cact_data.iterrows():
    t0 = row.iloc[1]
    t0 = t0.split()
    date = t0[0].split('/')
    t0 = t0[1].split(':')
    filename, swis_data = check_for_file(date)

    if swis_data is not None:
        if 'CME' not in swis_data.columns:
            swis_data['CME'] = 0
        if 'HALO' not in swis_data.columns:
            swis_data['HALO'] = 0
        # Get SWIS data and assign cme/halo columns
        start_time = datetime(int(date[0]), int(date[1]), int(date[2]), int(t0[0]), int(t0[1]))
        hr = int(t0[0]) + int(row.iloc[3])
        if hr > 23:
            hr = hr - 24
        end_time = datetime(int(date[0]), int(date[1]), int(date[2]), hr, int(t0[1]))
        for sno, obs in swis_data.iterrows():
            date_format = '%Y-%m-%d %H:%M:%S.%f'
            swis_time = datetime.strptime(obs.iloc[0], date_format)
            if swis_time > start_time and swis_time < end_time:
                swis_data.at[sno, 'CME'] = 1
                if row.iloc[-1] == "II":
                    swis_data.at[sno, 'HALO'] = 2
                elif row.iloc[-1] == "III":
                    swis_data.at[sno, 'HALO'] = 3
                elif row.iloc[-1] == "IV":
                    swis_data.at[sno, 'HALO'] = 4
                else:
                    swis_data.at[sno, 'HALO'] = 1

        # Save processed version
        output_file = os.path.join('/Users/vishrutgupta/Desktop/ISRO_bah_2025/merged_data', filename.replace('.cdf.csv', '.csv'))
        swis_data.to_csv(output_file, index=False)
        print(f"row {index} computed")

In [ ]:
### Combine multiple .csv files into one

file_paths = glob.glob("/Users/vishrutgupta/Desktop/ISRO_bah_2025/merged_data/*.csv")
merged_data = pd.DataFrame()

# Loop through each file and append data
for idx, file_path in enumerate(file_paths):
    df = pd.read_csv(file_path)
    if idx == 0:                    # skip header
        merged_data = df
    else:
        merged_data = pd.concat([merged_data, df], ignore_index=True)

merged_data = merged_data.sort_values(by=['epoch_for_cdf_mod'], ignore_index=True)
merged_data.to_csv("/Users/vishrutgupta/Desktop/ISRO_bah_2025/merged_data.csv", index=False)
